In [1]:
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

def filter_masks(masks, threshold):
    filter_masks = []
    total = masks[0]['segmentation'].shape[0] * masks[0]['segmentation'].shape[1]
    for i, mask_data in enumerate(masks):
        mask = mask_data["segmentation"]
        # filter
        total_value = np.sum(mask)
        percentage = total_value / total
        if percentage > threshold:
            continue
        filter_masks.append(mask_data)
    return filter_masks

## visual
def save_anns(image, anns, save_path):
    plt.figure(figsize=(20,20))
    plt.imshow(image)
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:,:,3] = 0
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.35]])
        img[m] = color_mask
    ax.imshow(img)
    plt.axis('off')
    plt.savefig(save_path)
    plt.close()

def write_masks_to_folder(image, masks, path, image_suffix, threshold=None):
    header = "id,area,bbox_x0,bbox_y0,bbox_w,bbox_h,point_input_x,point_input_y,predicted_iou,stability_score,crop_box_x0,crop_box_y0,crop_box_w,crop_box_h"  # noqa
    metadata = [header]
    # filter masks
    if threshold is not None:
        masks = filter_masks(masks, threshold)
    # save visual annotation fig
    save_anns(image, masks, os.path.join(path, 'full_mask'+image_suffix))
    # save each mask
    for i, mask_data in enumerate(masks):
        mask = mask_data["segmentation"]
        filename = f"{i}"+image_suffix
        cv2.imwrite(os.path.join(path, filename), mask * 255)
        mask_metadata = [
            str(i),
            str(mask_data["area"]),
            *[str(x) for x in mask_data["bbox"]],
            *[str(x) for x in mask_data["point_coords"][0]],
            str(mask_data["predicted_iou"]),
            str(mask_data["stability_score"]),
            *[str(x) for x in mask_data["crop_box"]],
        ]
        row = ",".join(mask_metadata)
        metadata.append(row)
    metadata_path = os.path.join(path, "metadata.csv")
    with open(metadata_path, "w") as f:
        f.write("\n".join(metadata))
    return

sam_checkpoint = '/home/a6/vis/LDS/SAM/weights/sam_vit_h_4b8939.pth'
model_type = 'vit_h'
device = 'cuda'
image_path = '/home/a6/vis/LDS/SAM/images/X2-G25m2-.png'

# create sam
sam_model = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam_model.to(device=device)
# sam_model.eval()
mask_generator = SamAutomaticMaskGenerator(
    model=sam_model,
    points_per_side=64,
    pred_iou_thresh=0.86,
    stability_score_thresh=0.75,
    crop_n_layers=2,
    crop_n_points_downscale_factor=2,
    min_mask_region_area=100,  # Requires open-cv to run post-processing
)

# load image
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image=cv2.resize(image,(1024,1024))

# 使用 matplotlib 展示图像
plt.imshow(image)
plt.title('Resized Image')
plt.axis('off')  # 隐藏坐标轴
plt.show()

masks = mask_generator.generate(image)
print(f'Number of masks generated: {len(masks)}')

# specify the directory to save masks
masks_dir = '/home/a6/vis/LDS/SAM/masks'
os.makedirs(masks_dir, exist_ok=True)

# save masks to the specified directory
write_masks_to_folder(image, masks, masks_dir, '.png', threshold=0.3)

Number of masks generated: 217


In [4]:
import json
import logging
import torch.nn.functional as F
import torch

# 创建 logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)

output_dir = '/home/a6/vis/LDS/SAM/outputs'
mask_vec_npy_dir = os.path.join(output_dir, 'npy_masks')
json_mask_avg_vec_save_path = os.path.join(output_dir, 'json_masks')
# Ensure output directories exist
os.makedirs(output_dir, exist_ok=True)
os.makedirs(mask_vec_npy_dir, exist_ok=True)
os.makedirs(json_mask_avg_vec_save_path, exist_ok=True)

image_torch = torch.as_tensor(image, device=device)
transformed_image = image_torch.permute(2, 0, 1).contiguous()[None, :, :, :]
input_image = sam_model.preprocess(transformed_image)
with torch.no_grad():
    image_embedding = sam_model.image_encoder(input_image)  # [B, C, H, W]
b, c, h, w = image_embedding.shape

# Process each mask
masked_feature_list = {}
mask_files = [f for f in os.listdir(masks_dir) if f.endswith('.png')]
mask_vecs = []
for mask_file in mask_files:
    mask_path = os.path.join(masks_dir, mask_file)
    mask_name = os.path.splitext(mask_file)[0]
    
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    mask = mask.astype(np.float32) / 255.0  # Normalize mask to [0, 1]
    mask = torch.as_tensor(mask, device=device)[None, None, :, :]  # [1, 1, H, W]
    
    # Rescale features and mask
    rescale_factor = 4  # 4x upsample/downsample
    t1, t2 = int(mask.shape[2] / rescale_factor), int(mask.shape[3] / rescale_factor)
    features_rescale = F.interpolate(image_embedding, size=[t1, t2], mode='bilinear')
    mask_rescale = F.interpolate(mask, size=[t1, t2], mode='bilinear')

    masked_feature = torch.mul(features_rescale, mask_rescale)
    masked_feature = masked_feature.view(b, c, -1)
    non_zero_count = torch.count_nonzero(masked_feature, dim=2)
    
    # Calculate average vector for mask
    masked_avg_vec = masked_feature.sum(dim=2) / non_zero_count
    masked_avg_vec[torch.isnan(masked_avg_vec)] = 0
    
    # Save masked feature vector
    npy_data = masked_avg_vec.detach().cpu().numpy()
    single_mask_vec_path = os.path.join(mask_vec_npy_dir, mask_name + '.npy')
    np.save(single_mask_vec_path, npy_data)
    masked_feature_list[mask_name] = single_mask_vec_path
    mask_vecs.append(npy_data)

# Save image and mask info to JSON
img_info = {
    'image_name': os.path.basename(image_path),
    'mask_avg_vec': masked_feature_list
}
json_save_name = os.path.join(json_mask_avg_vec_save_path, os.path.splitext(os.path.basename(image_path))[0] + '_mask_vec.json')
with open(json_save_name, "w") as file:
    json.dump(img_info, file)

logger.info(f'Saving image: {os.path.basename(image_path)} all masks as {json_save_name}.')



In [25]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import torch
from torchvision.ops import nms

# 加载所有 .npy 文件的特征向量
npy_dir = '/home/a6/vis/LDS/SAM/outputs/npy_masks'
npy_files = [f for f in os.listdir(npy_dir) if f.endswith('.npy')]
mask_dir = '/home/a6/vis/LDS/SAM/masks'

features = []
file_names = []

for npy_file in npy_files:
    file_path = os.path.join(npy_dir, npy_file)
    features.append(np.load(file_path))
    file_names.append(npy_file)

# 将特征向量转换为 NumPy 数组
features = np.array(features).squeeze()  # Assuming features are [N, D]

# 指定查询的文件名
query_file_name = "185.npy"  # 替换为你要查询的文件名
if query_file_name in file_names:
    query_index = file_names.index(query_file_name)
    query_feature = features[query_index]
else:
    raise ValueError(f"File {query_file_name} not found in the directory.")

# 计算相似性
similarities = cosine_similarity([query_feature], features)[0]

# 找到相似度大于0.75的特征向量
threshold = 0.5
similar_indices = np.where(similarities > threshold)[0]

# 输出相似度
print("Top similar vectors with similarity > 0.75:")
for idx in similar_indices:
    print(f"File: {file_names[idx]}, Similarity: {similarities[idx]}")

# 生成边界框并应用非极大抑制
bboxes = []
scores = []

for idx in similar_indices:
    mask_name = os.path.splitext(file_names[idx])[0] + '.png'
    mask_path = os.path.join(mask_dir, mask_name)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    y_indices, x_indices = np.where(mask > 0)
    if len(y_indices) == 0 or len(x_indices) == 0:
        continue
    x_min, x_max = np.min(x_indices), np.max(x_indices)
    y_min, y_max = np.min(y_indices), np.max(y_indices)
    bboxes.append([x_min, y_min, x_max, y_max])
    scores.append(similarities[idx])

# 转换为张量
bboxes = torch.tensor(bboxes, dtype=torch.float32)
scores = torch.tensor(scores, dtype=torch.float32)

# 应用非极大抑制
nms_indices = nms(bboxes, scores, iou_threshold=0.5)

# 过滤包含在更大掩码中的小掩码
kept_indices = []
for i in range(len(nms_indices)):
    keep = True
    for j in range(len(nms_indices)):
        if i != j:
            box_i = bboxes[nms_indices[i]]
            box_j = bboxes[nms_indices[j]]
            if (box_i[0] >= box_j[0] and box_i[1] >= box_j[1] and
                box_i[2] <= box_j[2] and box_i[3] <= box_j[3]):
                keep = False
                break
    if keep:
        kept_indices.append(nms_indices[i])

# 打印最终保留下来的掩码信息
print("Kept masks after NMS and containment filtering:")
for idx in kept_indices:
    mask_name = os.path.splitext(file_names[similar_indices[idx]])[0] + '.png'
    print(f"File: {mask_name}")

# 绘制复合掩码图像
image_shape = (1024, 1024)  # Assuming all masks are of this shape
composite_image = np.zeros(image_shape, dtype=np.float32)

for idx in kept_indices:
    mask_name = os.path.splitext(file_names[similar_indices[idx]])[0] + '.png'
    mask_path = os.path.join(mask_dir, mask_name)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE) / 255.0
    composite_image = np.maximum(composite_image, mask)

# 绘制复合掩码图像
plt.imshow(composite_image, cmap='gray')
plt.title('Composite Mask of Similar Features After NMS and Containment Filtering')
plt.show()


Top similar vectors with similarity > 0.75:
File: 118.npy, Similarity: 0.755237340927124
File: 131.npy, Similarity: 0.5630635023117065
File: 78.npy, Similarity: 0.7000281810760498
File: 82.npy, Similarity: 0.7385536432266235
File: 7.npy, Similarity: 0.8812510371208191
File: 25.npy, Similarity: 0.5241313576698303
File: 180.npy, Similarity: 0.5621994137763977
File: 201.npy, Similarity: 0.7745108604431152
File: 6.npy, Similarity: 0.8980224132537842
File: 190.npy, Similarity: 0.9171662330627441
File: 101.npy, Similarity: 0.7215646505355835
File: 186.npy, Similarity: 0.5510796904563904
File: 1.npy, Similarity: 0.5312522649765015
File: 155.npy, Similarity: 0.6397771835327148
File: 195.npy, Similarity: 0.5365434288978577
File: 94.npy, Similarity: 0.7076708674430847
File: 204.npy, Similarity: 0.6123093366622925
File: 8.npy, Similarity: 0.5210011601448059
File: 125.npy, Similarity: 0.5536914467811584
File: 185.npy, Similarity: 1.0
File: 176.npy, Similarity: 0.7368963956832886
File: 122.npy, Sim